In [1]:
%load_ext autoreload
%autoreload 2

%connect_info

{
  "shell_port": 51970,
  "shell_port": 58273,
  "iopub_port": 51971,
  "iopub_port": 59409,
  "stdin_port": 51972,
  "stdin_port": 46891,
  "control_port": 51974,
  "control_port": 54943,
  "hb_port": 51973,
  "hb_port": 59395,
  "ip": "127.0.0.1",
  "ip": "127.0.0.1",
  "key": "4ab6aeb9-54ce8c8ca0980c0b01674465",
  "ip": "127.0.0.1",
  "key": "c82274e4-fe78f677468ee816e97a1303",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-14cda08c-deac-4b74-a1ae-8bb80d5d491c.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
import django
from django.db import transaction

django.setup()

In [3]:
CONFIG_SHEET_NAME = "CCBP Configuration_v4"

In [4]:
from ib_iam.models import Project
Project.objects.create(project_id="JGC_DRIVE", display_id="jgc_drive", name="India 4.0")

<Project: Project object (JGC_DRIVE)>

### Populate project and project roles

In [5]:
@transaction.atomic()
def populate_project_and_roles():
    from ib_tasks.populate.population_utils import populate_project_roles
    populate_project_roles(CONFIG_SHEET_NAME)
    

@transaction.atomic()
def populate_task_and_boards_related_data():
    
    configuration_sheets = [
        CONFIG_SHEET_NAME
    ]

    for configuration_sheet in configuration_sheets:

        from ib_tasks.populate.population_utils import populate_data
        populate_data(configuration_sheet)
        
        from ib_boards.populate.get_data_from_sheet_for_boards_and_columns import GetBoardsAndColumnsDataFromSheet
        loader = GetBoardsAndColumnsDataFromSheet()
        loader.get_data_from_sheet(configuration_sheet)
        loader.create_boards_and_columns(configuration_sheet)

        
@transaction.atomic()
def populate_project_related_data():

    from ib_tasks.populate.population_utils import populate_projects_for_task_templates
    populate_projects_for_task_templates(CONFIG_SHEET_NAME)
    
    from ib_boards.populate.get_data_from_sheet_for_project_boards import GetSheetDataForProjectBoards
    loader = GetSheetDataForProjectBoards()
    loader.get_data_from_project_boards_sub_sheet(CONFIG_SHEET_NAME)

@transaction.atomic()
def create_indices_in_elasticsearch():
    from elasticsearch_dsl import connections
    from django.conf import settings
    es = connections.create_connection(
        hosts=[settings.ELASTICSEARCH_ENDPOINT], timeout=20
    )

    from ib_tasks.documents.elastic_task import TASK_INDEX_NAME
    es.indices.create(index=TASK_INDEX_NAME)



    
    
@transaction.atomic()
def load_task_template_from_sheet():
#     create_indices_in_elasticsearch()÷
        
    populate_project_and_roles()
    populate_task_and_boards_related_data()
    populate_project_related_data()


In [6]:
load_task_template_from_sheet()

[BoardDTO(board_id='CCBPB_LEADS', name='India 4.0 Beneficiaries Calls'), BoardDTO(board_id='CCBPB_LEADS_STATUS', name='India 4.0 Beneficiaries Status')] [ColumnDTO(column_id='CCBPC_DRAFTS', name='Beneficiaries Drafts', display_order=1, task_template_stages=[TaskTemplateStagesDTO(task_template_id='CCBP_LGA', stages=['CCBP_DRAFTS'])], user_role_ids=['ALL_ROLES'], column_summary='ColumnSummary1', column_actions='ColumnSummary3', list_view_fields=[TaskSummaryFieldsDTO(task_id='ravi', summary_fields='ravi')], kanban_view_fields=[TaskSummaryFieldsDTO(task_id='ravi', summary_fields='ravi')], board_id='CCBPB_LEADS'), ColumnDTO(column_id='CCBP_LEADS', name='Beneficiaries', display_order=2, task_template_stages=[TaskTemplateStagesDTO(task_template_id='CCBP_LGA', stages=['CCBP_LEADS'])], user_role_ids=['ALL_ROLES'], column_summary='ColumnSummary1', column_actions='ColumnSummary3', list_view_fields=[TaskSummaryFieldsDTO(task_id='ravi', summary_fields='ravi')], kanban_view_fields=[TaskSummaryFields